In [1]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from scipy import spatial
import networkx as nx

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\irfan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\irfan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Open AI_summarization.txt article
file = open("AI_summarization.txt", "r")
text = file.read()

In [12]:
# Open exercise.txt article
file = open("exercise.txt", "r")
text = file.read()

In [62]:
# Open chatgpt.txt article
file = open("chatgpt.txt", "r")
text = file.read()

In [41]:
# Open time_travel.txt article
file = open("time_travel.txt", "r")
text = file.read()

In [52]:
# Open confess.txt article
file = open("confess.txt", "r")
text = file.read()

In [63]:
Sentences=sent_tokenize(text)

In [64]:
length = len(Sentences)
summary_length = int(length/5)

length_str = str(length)
sum_length_str = str(summary_length)

print("Total sentences: " + length_str)
print("Summary sentence: " + sum_length_str)

Sentences


Total sentences: 17
Summary sentence: 3


['The future of ChatGPT is a topic of much excitement and speculation in the technology and AI communities.',
 'As one of the largest language models developed by OpenAI, ChatGPT has already had a significant impact on the field of natural language processing and has the potential to change the way we interact with computers and technology.',
 'In the near future, it is likely that ChatGPT will continue to improve and become even more advanced.',
 'As more data and computational power becomes available, the model will be able to process and understand an even larger range of languages, topics, and styles of writing.',
 'This will make it even more effective for a variety of tasks, such as language translation, question-answering, and text generation.',
 'One of the most exciting applications of ChatGPT in the future is its potential to revolutionize the field of customer service.',
 'With its ability to understand and respond to customer inquiries in natural language, ChatGPT could rep

In [65]:
sentences_clean=[re.sub(r'[^\w\s]','',sentence.lower()) for sentence in Sentences]
stop_words = stopwords.words('english')
sentence_tokens=[[words for words in sentence.split(' ') if words not in stop_words] for sentence in sentences_clean]

In [66]:
sentence_tokens

[['future',
  'chatgpt',
  'topic',
  'much',
  'excitement',
  'speculation',
  'technology',
  'ai',
  'communities'],
 ['one',
  'largest',
  'language',
  'models',
  'developed',
  'openai',
  'chatgpt',
  'already',
  'significant',
  'impact',
  'field',
  'natural',
  'language',
  'processing',
  'potential',
  'change',
  'way',
  'interact',
  'computers',
  'technology'],
 ['near',
  'future',
  'likely',
  'chatgpt',
  'continue',
  'improve',
  'become',
  'even',
  'advanced'],
 ['data',
  'computational',
  'power',
  'becomes',
  'available',
  'model',
  'able',
  'process',
  'understand',
  'even',
  'larger',
  'range',
  'languages',
  'topics',
  'styles',
  'writing'],
 ['make',
  'even',
  'effective',
  'variety',
  'tasks',
  'language',
  'translation',
  'questionanswering',
  'text',
  'generation'],
 ['one',
  'exciting',
  'applications',
  'chatgpt',
  'future',
  'potential',
  'revolutionize',
  'field',
  'customer',
  'service'],
 ['ability',
  'und

In [67]:
model = Word2Vec(sentence_tokens, vector_size=1, min_count=1, epochs=2000)

sentence_embeddings = []
for words in sentence_tokens:
    sentence_embedding = np.mean([model.wv[word] for word in words], axis=0)
    sentence_embeddings.append(sentence_embedding)

similarity_matrix = np.zeros([len(sentence_tokens), len(sentence_tokens)])

for i, row_embedding in enumerate(sentence_embeddings):
    for j, column_embedding in enumerate(sentence_embeddings):
        similarity_matrix[i][j] = 1 - spatial.distance.cosine(row_embedding, column_embedding)

In [68]:
nx_graph = nx.from_numpy_array(similarity_matrix)
scores = nx.pagerank(nx_graph)

In [69]:
top_sentence={sentence:scores[index] for index,sentence in enumerate(Sentences)}
top=dict(sorted(top_sentence.items(), key=lambda x: x[1], reverse=True)[:summary_length])

In [70]:
top

{'By integrating with other AI technologies, such as computer vision and robotics, ChatGPT could form the foundation of a new class of intelligent machines that can interact with us in a natural and intuitive way.': 0.058823530442986965,
 'One of the biggest challenges will be to ensure that the model is transparent, ethical, and trustworthy.': 0.058823530092371394,
 'This will require ongoing research and development to address issues such as bias, accountability, and privacy.': 0.058823530092371394}

In [71]:
for sent in Sentences:
    if sent in top.keys():
        print(sent)

By integrating with other AI technologies, such as computer vision and robotics, ChatGPT could form the foundation of a new class of intelligent machines that can interact with us in a natural and intuitive way.
One of the biggest challenges will be to ensure that the model is transparent, ethical, and trustworthy.
This will require ongoing research and development to address issues such as bias, accountability, and privacy.
